In [70]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import fastai 
from pathlib import Path
import fastai.io as io
import numpy as np

# SETUP

In [2]:
path=Path('data/nietzsche')
path.mkdir(exist_ok=True)

In [3]:
str(path)

'data/nietzsche'

# Data

In [4]:
io.get_data("https://s3.amazonaws.com/text-datasets/nietzsche.txt", path/'nietzsche.txt')

nietzsche.txt: 606kB [00:01, 454kB/s]                             


In [11]:
file = open(path/'nietzsche.txt', encoding='utf-8')
text = file.read()

In [12]:
len(text)

600893

## chars, chars <-> corpus_idx 

In [51]:
chars = sorted(list(set(text))); 
vocab_size = len(chars) +1; 
chars.insert(0, "\0")

In [52]:
''.join(chars) # this is our token set

'\x00\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzÆäæéë'

In [53]:
char_indices = { c: i for i, c in enumerate(chars)}
indices_char = { i: c for i, c in enumerate(chars)}

In [54]:
indices_char[char_indices['a']]

'a'

### Corpus token -> char set

In [62]:
idx = [char_indices[c] for c in text]

In [63]:
bptt = 3

In [68]:
c1_data = [idx[i] for i in range(0, len(idx)-bptt, bptt) ]
c2_data = [idx[i+1] for i in range(0, len(idx)-bptt, bptt) ]
c3_data = [idx[i+2] for i in range(0, len(idx)-bptt, bptt) ]
c4_data = [idx[i+3] for i in range(0, len(idx)-bptt, bptt) ]

Inputs

In [80]:
x1 = np.stack(c1_data)
x2 = np.stack(c2_data)
x3 = np.stack(c3_data)

outputs

In [81]:
y = np.stack(c4_data)

In [82]:
x1[:4], x2[:4], x3[:4]

(array([40, 30, 29,  1]), array([42, 25,  1, 43]), array([29, 27,  1, 45]))

In [83]:
y[:4]

array([30, 29,  1, 40])

## nn.Module